### 문제 2-1 : 콤마 구분 리스트 파서 활용
#### CommaSeparatedListOutputParser

In [ ]:
# 필요한 라이브러리 설치
#%pip install -q langchain langchain-openai python-dotenv

In [1]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [2]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [3]:
# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""
    사용자가 관심있는 분야는 '{subject}'입니다.
    이 분야와 관련된 한국의 유명한 장소나 활동 5가지를 추천해 주세요.
    
    {format_instructions}
    """,
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

# 모델 설정 (Groq API 사용)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 생성
chain = prompt | model | output_parser

# 테스트 실행
test_subjects = ["음식", "스포츠", "역사", "자연"]

print("=== 한국 관련 추천 장소/활동 ===\n")

for subject in test_subjects:
    try:
        result = chain.invoke({"subject": subject})
        print(f" 분야: {subject}")
        print(f" 추천 목록: {result}")
        print(f" 개수: {len(result)}개")
        print("-" * 50)
    except Exception as e:
        print(f" {subject} 처리 중 오류: {e}")

# 사용자 입력 받기 (선택사항)
print("\n 직접 입력해보세요!")
user_input = input("관심 분야를 입력하세요: ")
if user_input:
    try:
        result = chain.invoke({"subject": user_input})
        print(f"\n '{user_input}' 관련 추천:")
        for i, item in enumerate(result, 1):
            print(f"  {i}. {item}")
    except Exception as e:
        print(f" 처리 중 오류: {e}")

=== 한국 관련 추천 장소/활동 ===

 분야: 음식
 추천 목록: ['서울의 한정식 거리', '부산의 자갈치 시장', '전주 음식 마을', '경주의 황남빵 마을', '남해의 멍게비빔밥 거리']
 개수: 5개
--------------------------------------------------
 분야: 스포츠
 추천 목록: ['서울 올림픽공원', '수원 월드컵경기장', '부산 아시아드주경기장', '평창 알펜시아 스키점프장', '인천 선학 국제도시 롱보드 다운힐 코스']
 개수: 5개
--------------------------------------------------
 분야: 역사
 추천 목록: ['경복궁', '수원 화성', '독립기념관', '국립중앙박물관', '강화도 역사 둘러보기']
 개수: 5개
--------------------------------------------------
 분야: 자연
 추천 목록: ['설악산', '한라산', '제주도 올레길', '경주 국립공원', '부산 태종대']
 개수: 5개
--------------------------------------------------

 직접 입력해보세요!

 '자동차' 관련 추천:
  1. 자동차 테마파크
  2. 현대자동차 공장 견학
  3. 기아자동차 광주 공장
  4. 부산 국제 자동차 전시회
  5. 자동차 튜닝 거리


### 문제 2-2 : 영화 리뷰 감정 분석기
### 출력 파서: EnumOutputParser


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from enum import Enum

In [5]:
# 감정 클래스 정의 (Enum)
class MovieSentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=MovieSentiment)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
당신은 영화 리뷰 감정 분석 전문가입니다.
다음 영화 리뷰의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

리뷰: "{review}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 오직 하나의 단어만 출력하세요

답변:
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

# 안전한 감정 분석 함수
def analyze_movie_sentiment(review_text):
    """영화 리뷰 감정 분석 함수"""
    try:
        chain = prompt | model | fixing_parser
        result = chain.invoke({"review": review_text})
        return result, None
    except Exception as e:
        return None, f"오류: {str(e)}"

# 테스트 리뷰 데이터
test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
    "최고의 영화! 꼭 보세요!",
    "너무 지루하고 스토리가 말이 안 돼요.",
    "볼만한 영화입니다. 시간 때우기에는 좋네요."
]

print("=== 영화 리뷰 감정 분석 결과 ===\n")

success_count = 0
total_count = len(test_reviews)

for i, review in enumerate(test_reviews, 1):
    print(f"{i}. 리뷰: {review}")
    
    result, error = analyze_movie_sentiment(review)
    
    if result:
        # 감정에 따른 이모지 추가
        emotion_emoji = {
            "긍정": "😊",
            "부정": "😞", 
            "보통": "😐"
        }
        emoji = emotion_emoji.get(result.value, "❓")
        
        print(f"   감정: {result.value} {emoji}")
        success_count += 1
    else:
        print(f"   오류: {error}")
    
    print("-" * 60)

# 결과 요약
print(f"\n=== 분석 결과 요약 ===")
print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
print(f"실패: {total_count-success_count}/{total_count}")

# 감정별 통계 (성공한 케이스만)
if success_count > 0:
    print(f"\n=== 감정 분포 ===")
    sentiment_count = {"긍정": 0, "부정": 0, "보통": 0}
    
    for review in test_reviews:
        result, error = analyze_movie_sentiment(review)
        if result:
            sentiment_count[result.value] += 1
    
    for sentiment, count in sentiment_count.items():
        percentage = (count / success_count * 100) if success_count > 0 else 0
        print(f"{sentiment}: {count}개 ({percentage:.1f}%)")


=== 영화 리뷰 감정 분석 결과 ===

1. 리뷰: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 😞
------------------------------------------------------------
2. 리뷰: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 😊
------------------------------------------------------------
3. 리뷰: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 😐
------------------------------------------------------------
4. 리뷰: 최고의 영화! 꼭 보세요!
   감정: 긍정 😊
------------------------------------------------------------
5. 리뷰: 너무 지루하고 스토리가 말이 안 돼요.
   감정: 부정 😞
------------------------------------------------------------
6. 리뷰: 볼만한 영화입니다. 시간 때우기에는 좋네요.
   감정: 보통 😐
------------------------------------------------------------

=== 분석 결과 요약 ===
성공: 6/6 (100.0%)
실패: 0/6

=== 감정 분포 ===
긍정: 2개 (33.3%)
부정: 2개 (33.3%)
보통: 2개 (33.3%)


In [6]:

# 사용자 입력 테스트 (선택사항)
print("\n 직접 리뷰를 입력해보세요!")
user_review = input("영화 리뷰를 입력하세요: ")
if user_review.strip():
    result, error = analyze_movie_sentiment(user_review)
    if result:
        emotion_emoji = {"긍정": "😊", "부정": "😞", "보통": "😐"}
        emoji = emotion_emoji.get(result.value, "❓")
        print(f"\n 분석 결과: {result.value} {emoji}")
    else:
        print(f"\n 분석 실패: {error}")


 직접 리뷰를 입력해보세요!

 분석 결과: 부정 😞


### 문제 2-3: 학생 정보 구조화 시스템
### 출력 파서: PydanticOutputParser

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
import json
from dotenv import load_dotenv

# 학생 정보를 정의하는 Pydantic 모델
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 목록")
    goal: str = Field(description="학생의 목표나 꿈")

# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 프롬프트 템플릿 설정
template = """
다음 학생의 자기소개에서 정보를 추출하여 구조화 해주세요.

자기소개: {introduction}

{format_instructions}

중요사항:
- 이름, 나이, 전공, 취미, 목표를 정확히 추출하세요
- 취미는 리스트 형태로 여러 개 추출하세요
- 명시되지 않은 정보는 "정보 없음"으로 표시하세요
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3  # 정확한 정보 추출을 위해 낮은 온도
)

# 체인 구성
chain = prompt | model | parser

# 학생 정보 추출 함수
def extract_student_info(introduction_text):
    """학생 자기소개에서 정보를 추출하는 함수"""
    try:
        result = chain.invoke({"introduction": introduction_text})
        return result, None
    except Exception as e:
        return None, f"오류: {str(e)}"

# 테스트 자기소개 데이터
test_introductions = [
    "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.",
    
    "제 이름은 박지연이고, 올해 20세입니다. 경영학과에 다니고 있고, 독서와 여행, 요리를 즐깁니다. 미래에는 글로벌 기업의 CEO가 되고 싶어요.",
    
    "이수진입니다. 나이는 23살이고 미술을 전공하고 있습니다. 그림 그리기, 사진 찍기, 전시회 관람이 취미예요. 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.",
    
    "안녕하세요~ 저는 최진우예요. 21살이고 기계공학과 학생입니다. 축구, 농구 같은 운동을 좋아하고 음악 듣기도 즐겨해요. 앞으로 로봇 엔지니어가 되는 게 꿈이에요."
]

print("=== 학생 정보 구조화 시스템 ===\n")

for i, introduction in enumerate(test_introductions, 1):
    print(f" 테스트 케이스 {i}")
    print(f"자기소개: {introduction}")
    print()
    
    # 정보 추출
    student_info, error = extract_student_info(introduction)
    
    if student_info:
        print(" 추출된 정보:")
        print(f" 이름: {student_info.name}")
        print(f" 나이: {student_info.age}세")
        print(f" 전공: {student_info.major}")
        print(f" 취미: {', '.join(student_info.hobbies)}")
        print(f" 목표: {student_info.goal}")
        
        # JSON 형태로도 출력
        print("\n JSON 형태:")
        student_dict = student_info.dict()
        print(json.dumps(student_dict, ensure_ascii=False, indent=2))
        
    else:
        print(f" 추출 실패: {error}")
    
    print("="*80)
    print()


=== 학생 정보 구조화 시스템 ===

 테스트 케이스 1
자기소개: 안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다.

 추출된 정보:
 이름: 김민수
 나이: 22세
 전공: 컴퓨터공학
 취미: 게임하기, 영화보기, 코딩
 목표: 훌륭한 개발자가 되는 것

 JSON 형태:
{
  "name": "김민수",
  "age": 22,
  "major": "컴퓨터공학",
  "hobbies": [
    "게임하기",
    "영화보기",
    "코딩"
  ],
  "goal": "훌륭한 개발자가 되는 것"
}

 테스트 케이스 2
자기소개: 제 이름은 박지연이고, 올해 20세입니다. 경영학과에 다니고 있고, 독서와 여행, 요리를 즐깁니다. 미래에는 글로벌 기업의 CEO가 되고 싶어요.

 추출된 정보:
 이름: 박지연
 나이: 20세
 전공: 경영학과
 취미: 독서, 여행, 요리
 목표: 글로벌 기업의 CEO가 되고 싶어요

 JSON 형태:
{
  "name": "박지연",
  "age": 20,
  "major": "경영학과",
  "hobbies": [
    "독서",
    "여행",
    "요리"
  ],
  "goal": "글로벌 기업의 CEO가 되고 싶어요"
}

 테스트 케이스 3
자기소개: 이수진입니다. 나이는 23살이고 미술을 전공하고 있습니다. 그림 그리기, 사진 찍기, 전시회 관람이 취미예요. 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.

 추출된 정보:
 이름: 이수진
 나이: 23세
 전공: 미술
 취미: 그림 그리기, 사진 찍기, 전시회 관람
 목표: 유명한 작가가 되어 많은 사람들에게 감동을 주고 싶습니다.

 JSON 형태:
{
  "name": "이수진",
  "age": 23,
  "major": "미술",
  "hobbies": [
    "그림 그리기",
    "사진 찍기",
   

In [8]:

# 사용자 입력 테스트 (선택사항)
print(" 직접 자기소개를 입력해보세요!")
user_introduction = input("자기소개를 입력하세요: ")

if user_introduction.strip():
    print("\n 정보 추출 중...")
    student_info, error = extract_student_info(user_introduction)
    
    if student_info:
        print("\n 추출 결과:")
        print(f" 이름: {student_info.name}")
        print(f" 나이: {student_info.age}세")
        print(f" 전공: {student_info.major}")
        print(f" 취미: {', '.join(student_info.hobbies)}")
        print(f" 목표: {student_info.goal}")
        
        # 정보 검증
        print(f"\n 추출된 취미 개수: {len(student_info.hobbies)}개")
        if len(student_info.hobbies) >= 3:
            print(" 충분한 취미 정보가 추출되었습니다!")
        
    else:
        print(f"\n 추출 실패: {error}")

print("\n 학생 정보 구조화 완료!")

# 추가 기능: 통계 정보
print("\n 테스트 케이스 통계:")
majors = []
total_hobbies = []

for introduction in test_introductions:
    student_info, error = extract_student_info(introduction)
    if student_info:
        majors.append(student_info.major)
        total_hobbies.extend(student_info.hobbies)

if majors:
    print(f"전공 분포: {', '.join(set(majors))}")
    print(f"총 취미 종류: {len(set(total_hobbies))}개")
    print(f"가장 인기 있는 취미: {max(set(total_hobbies), key=total_hobbies.count)}")

 직접 자기소개를 입력해보세요!

 학생 정보 구조화 완료!

 테스트 케이스 통계:


NameError: name 'test_introductions' is not defined

### 문제 2-4 : 여행 계획 분석기
### 출력 파서: StructuredOutputParser

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json
from pprint import pprint
from dotenv import load_dotenv

# 출력 구조 정의 (여행 정보 추출)
response_schemas = [
    ResponseSchema(name="destination", description="여행 목적지 또는 지역"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일, 1주일 등)"),
    ResponseSchema(name="budget", description="여행 예산 (원 단위 또는 대략적인 금액)"),
    ResponseSchema(name="rating", description="여행 만족도 또는 추천도 (1-5점 척도)"),
    ResponseSchema(name="activities", description="주요 활동이나 관광지 목록을 문자열로 (콤마로 구분)")
]

# StructuredOutputParser 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
다음 여행 후기나 계획에서 핵심 정보를 추출하여 구조화해주세요.

여행 후기/계획: {travel_text}

{format_instructions}

중요사항:
- 여행지는 구체적인 지명으로 추출하세요
- 기간은 "N박 N일" 또는 "N일" 형태로 표준화하세요  
- 예산은 숫자와 함께 "만원" 단위로 표시하세요
- 평점은 1-5점 사이의 숫자만 표시하세요
- 활동은 콤마로 구분된 문자열 형태로 작성하세요
- 명시되지 않은 정보는 "정보 없음"으로 표시하세요
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3  # 정확한 정보 추출을 위해 낮은 온도
)

# 체인 구성
chain = prompt | model | parser

# 여행 정보 추출 함수
def extract_travel_info(travel_text):
    """여행 후기/계획에서 정보를 추출하는 함수"""
    try:
        result = chain.invoke({"travel_text": travel_text})
        return result, None
    except Exception as e:
        return None, f"오류: {str(e)}"

# 테스트 여행 후기/계획 데이터
test_travel_texts = [
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.",
    
    "이번 여름에 제주도로 3박 4일 계획하고 있어요. 예산은 50만원 정도 생각하고 있고, 성산일출봉 등반, 한라산 트레킹, 우도 관광, 흑돼지 맛집 투어를 할 예정입니다. 아직 가지 않았지만 기대가 많이 돼서 5점 드리고 싶어요!",
    
    "강릉으로 1박 2일 다녀왔습니다. 경포대 해변 산책하고 커피거리에서 커피 마시고 오죽헌도 구경했어요. 20만원 정도 들었는데 그냥 무난했어요. 3점 정도?",
    
    "서울 3일 여행! 경복궁, 명동 쇼핑, 한강 공원 피크닉, 홍대 클럽 체험했어요. 40만원 사용했고 너무 재밌었어요. 완전 5점!",
    
    "전주 한옥마을 당일치기 여행했어요. 비빔밥 먹고 한복 체험하고 전동성당 구경했습니다. 15만원 정도 썼고 괜찮았어요. 4점 정도 드릴게요."
]

print("=== 여행 계획 분석기 ===\n")

for i, travel_text in enumerate(test_travel_texts, 1):
    print(f"\ 테스트 케이스 {i}")
    print(f"여행 후기/계획: {travel_text}")
    print()
    
    # 여행 정보 추출
    travel_info, error = extract_travel_info(travel_text)
    
    if travel_info:
        print(" 추출된 여행 정보:")
        print(f"  여행지: {travel_info['destination']}")
        print(f"  기간: {travel_info['duration']}")
        print(f"  예산: {travel_info['budget']}")
        print(f"  평점: {travel_info['rating']}점")
        print(f"  주요 활동: {travel_info['activities']}")
        
        # JSON 형태로도 출력
        print("\n JSON 형태:")
        print(json.dumps(travel_info, ensure_ascii=False, indent=2))
        
        # 활동 개수 카운트
        if travel_info['activities'] != "정보 없음":
            activities_list = [activity.strip() for activity in travel_info['activities'].split(',')]
            print(f"\n 총 활동 개수: {len(activities_list)}개")
        
    else:
        print(f" 추출 실패: {error}")
    
    print("="*80)
    print()

<>:76: SyntaxWarning: invalid escape sequence '\ '
<>:76: SyntaxWarning: invalid escape sequence '\ '
C:\Users\user\AppData\Local\Temp\ipykernel_16668\2676565429.py:76: SyntaxWarning: invalid escape sequence '\ '
  print(f"\ 테스트 케이스 {i}")


=== 여행 계획 분석기 ===

\ 테스트 케이스 1
여행 후기/계획: 지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요.

 추출된 여행 정보:
  여행지: 부산
  기간: 2박3일
  예산: 30만원
  평점: 4점
  주요 활동: 해운대, 자갈치시장, 감천문화마을

 JSON 형태:
{
  "destination": "부산",
  "duration": "2박3일",
  "budget": "30만원",
  "rating": "4",
  "activities": "해운대, 자갈치시장, 감천문화마을"
}

 총 활동 개수: 3개

\ 테스트 케이스 2
여행 후기/계획: 이번 여름에 제주도로 3박 4일 계획하고 있어요. 예산은 50만원 정도 생각하고 있고, 성산일출봉 등반, 한라산 트레킹, 우도 관광, 흑돼지 맛집 투어를 할 예정입니다. 아직 가지 않았지만 기대가 많이 돼서 5점 드리고 싶어요!

 추출된 여행 정보:
  여행지: 제주도
  기간: 3박4일
  예산: 50만원
  평점: 5점
  주요 활동: 성산일출봉 등반, 한라산 트레킹, 우도 관광, 흑돼지 맛집 투어

 JSON 형태:
{
  "destination": "제주도",
  "duration": "3박4일",
  "budget": "50만원",
  "rating": "5",
  "activities": "성산일출봉 등반, 한라산 트레킹, 우도 관광, 흑돼지 맛집 투어"
}

 총 활동 개수: 4개

\ 테스트 케이스 3
여행 후기/계획: 강릉으로 1박 2일 다녀왔습니다. 경포대 해변 산책하고 커피거리에서 커피 마시고 오죽헌도 구경했어요. 20만원 정도 들었는데 그냥 무난했어요. 3점 정도?

 추출된 여행 정보:
  여행지: 강릉
  기간: 1박 2일
  예산: 20만원
  평점: 3점
  주요 활동: 경포대 해변, 

In [ ]:

# 사용자 입력 테스트 (선택사항)
print(" 직접 여행 후기나 계획을 입력해보세요!")
user_travel_text = input("여행 후기/계획을 입력하세요: ")

if user_travel_text.strip():
    print("\n 여행 정보 추출 중...")
    travel_info, error = extract_travel_info(user_travel_text)
    
    if travel_info:
        print("\n 추출 결과:")
        print(f" 여행지: {travel_info['destination']}")
        print(f" 기간: {travel_info['duration']}")
        print(f" 예산: {travel_info['budget']}")
        print(f" 평점: {travel_info['rating']}점")
        print(f" 주요 활동: {travel_info['activities']}")
        
        # 간단한 분석
        try:
            rating = float(travel_info['rating'])
            if rating >= 4.0:
                print(" 높은 만족도의 여행이네요!")
            elif rating >= 3.0:
                print(" 괜찮은 여행이었군요!")
            else:
                print(" 아쉬운 부분이 있었나보네요.")
        except:
            print(" 평점 정보를 분석할 수 없습니다.")
        
    else:
        print(f"\n 추출 실패: {error}")

# 추가 기능: 여행 통계 분석
print("\n 테스트 케이스 통계 분석:")

destinations = []
total_activities = []
ratings = []
budgets = []

for travel_text in test_travel_texts:
    travel_info, error = extract_travel_info(travel_text)
    if travel_info:
        if travel_info['destination'] != "정보 없음":
            destinations.append(travel_info['destination'])
        
        if travel_info['activities'] != "정보 없음":
            activities_list = [activity.strip() for activity in travel_info['activities'].split(',')]
            total_activities.extend(activities_list)
        
        try:
            rating = float(travel_info['rating'])
            ratings.append(rating)
        except:
            pass
        
        if travel_info['budget'] != "정보 없음":
            budgets.append(travel_info['budget'])

if destinations:
    print(f" 방문 지역: {', '.join(set(destinations))}")

if ratings:
    avg_rating = sum(ratings) / len(ratings)
    print(f" 평균 만족도: {avg_rating:.1f}점")

if total_activities:
    print(f" 총 활동 종류: {len(set(total_activities))}개")
    most_popular = max(set(total_activities), key=total_activities.count)
    print(f" 가장 인기 활동: {most_popular}")

if budgets:
    print(f" 예산 범위: {', '.join(set(budgets))}")

print("\n 여행 계획 분석 완료!")

# 여행 추천 기능 (보너스)
def recommend_travel_style(rating, activities_count):
    """평점과 활동 수를 바탕으로 여행 스타일 추천"""
    if rating >= 4.5 and activities_count >= 4:
        return " 액티브 어드벤처형 - 다양한 활동을 즐기는 여행스타일"
    elif rating >= 4.0 and activities_count <= 3:
        return " 힐링 휴식형 - 여유롭게 즐기는 여행스타일"
    elif rating >= 3.0:
        return " 일반 관광형 - 무난하게 즐기는 여행스타일"
    else:
        return " 신중 계획형 - 더 신중한 계획이 필요한 스타일"

print("\n 여행 스타일 분석:")
for i, travel_text in enumerate(test_travel_texts[:3], 1):  # 처음 3개만 분석
    travel_info, error = extract_travel_info(travel_text)
    if travel_info:
        try:
            rating = float(travel_info['rating'])
            activities_list = [activity.strip() for activity in travel_info['activities'].split(',')]
            activities_count = len(activities_list)
            
            style = recommend_travel_style(rating, activities_count)
            print(f"{i}. {travel_info['destination']} 여행: {style}")
        except:
            print(f"{i}. {travel_info['destination']} 여행: 분석 불가")

print("\n 모든 분석이 완료되었습니다!") 